In [2]:
import time
import pymongo
import json
import os
import tqdm
import re
import openai
import tiktoken
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from deep_translator import GoogleTranslator # До 5000 символов за раз

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
transtator = GoogleTranslator(source='ru', target='en')

In [4]:
client_mongo = pymongo.MongoClient('mongodb://localhost:27017/')
db_hr = client_mongo.hr
vacancies_collection = db_hr.vacancies
resumes_collection = db_hr.resumes

In [2]:
# Записываю в коллекцию vacancies базы hr все вакансии из файлов json
for filename in os.listdir('../Vacancies'): # Локальная папка с вакансиями
    if filename.endswith('.json'):
        with open(os.path.join('../Vacancies', filename), 'r', encoding='utf-8') as v:
            vacancy_dict = json.load(v)
            vacancy_dict['_id'] = filename # Добавляю поле "_id" с именем файла для каждой вакансии
            try:
                vacancies_collection.insert_one(vacancy_dict)
            except Exception as ex:
                pass

# Записываю в коллекцию resumes базы hr все резюме из файлов json
for filename in os.listdir('../Resumes'): # Локальная папка с резюме
    if filename.endswith('.json'):
        with open(os.path.join('../Resumes', filename), 'r', encoding='utf-8') as r:
            resume_dict = json.load(r)
            resume_dict['_id'] = filename # Добавляю поле "_id" с именем файла для каждого резюме
            try:
                resumes_collection.insert_one(resume_dict)
            except Exception as ex:
                pass

print(f'К-во вакансий в базе данных: {vacancies_collection.count_documents({})}. '
      f'\nК-во резюме в базе данных: {resumes_collection.count_documents({})}.')

К-во вакансий в базе данных: 100. 
К-во резюме в базе данных: 100.


# Вакансии
### Ключевые поля (основные требования) определям из вакансий
### Ключевые поля:
- Позиция "position"
- Уровень "experienceLevels"
- Формат работы "workFormat"
- Локация "requiredLocation"
- Зарплата "partnerRates" в час (168 часов в месяце)
- Проектные задачи "projectTasks"
#### Три поля по скилам:
- Скилы "skills"
- Требования "mandatoryRequirements"
- Доп. требования "additionalRequirements"

In [16]:
# Формирую строки из ключевых полей
for value in tqdm.tqdm(vacancies_collection.find()): # по всем вакансиям из коллекции vacancies

    try: # Если указана зарплата (за час)
        salary = int(re.findall(r'\d+', f"{value['data']['partnerRates']}")[0]) * 168 # Зарплата за 168 часов в месяц
        salary = f"Salary: {salary} рублей в месяц."
    except:
        salary = ''

    try: # Если указаны Add.Requirements
        add_requirements = f"Add.Requirements: {(' ').join(value['data']['additionalRequirements'])}"
    except:
        add_requirements = ''

    # Position, Skills, M.Requirements, Add.Requirements *******************************
    vacancy_line_1 = re.sub(r'\s+', ' ', f"""
                   Position: {value['data']['position']}.
                   Skills: {(', ').join(value['skills'])}.
                   Requirements: {(' ').join(value['data']['mandatoryRequirements'])}
                   {add_requirements}""")
    vacancy_line_1_en = transtator.translate(vacancy_line_1)
    time.sleep(1)

    # Levels, WorkFormat, Location, Salary **********************************************
    vacancy_line_2 = re.sub(r'\s+', ' ', f"""
                   Experience Levels: {(', ').join(value['data']['experienceLevels'])}.
                   Work Format: {value['data']['workFormat']}.
                   Required Location: {value['data']['requiredLocation']}.
                   {salary}""")
    vacancy_line_2_en = transtator.translate(vacancy_line_2)
    time.sleep(1)

    try: # Project Tasks ****************************************************************
        vacancy_line_3 = re.sub(r'\s+', ' ', f"Tasks: {(' ').join(value['data']['projectTasks'])}")
        vacancy_line_3_en = transtator.translate(vacancy_line_3)
        time.sleep(1)
    except:
        vacancy_line_3 = ''
        vacancy_line_3_en = ''

    try: # сохраняю сформированные строки из полей вакансий в коллекцию vacancies
        vacancies_collection.find_one_and_update({'_id': value['_id']}, # нахожу вакансию по _id
                                                 {'$set': {'line_1': vacancy_line_1,
                                                           'line_1_en': vacancy_line_1_en,
                                                           'line_2': vacancy_line_2,
                                                           'line_2_en': vacancy_line_2_en,
                                                           'line_3': vacancy_line_3,
                                                           'line_3_en': vacancy_line_3_en}})
    except Exception as ex:
        print(ex)

# Пример строки с выбранными ключевыми полями
ex_vacancy = vacancies_collection.find_one({'_id': 'Kotlin_разработчик-6bcde28f-d5b1-4586-82f2-9731cd38c756.json'})
print(ex_vacancy['line_1_en'], '\n')
print(ex_vacancy['line_2_en'], '\n')
print(ex_vacancy['line_3_en'], '\n')

100it [15:16,  9.16s/it]

Position: Kotlin developer. Skills: Accounting, Management accounting, The development of the software, Setting the tasks to developers, Remote work, Optimizing the query, PostgreSQL is, The system, The CI/CD, The SOLID, The Kotlin, Docker, The Nexus, The CI , Unit Testing, The Kotlin Coroutines, MVC, ETL, The system analysis, ETL of the process, Big data, Video processing, solid, Agile Project Management, The financial analysis, Project documentation, Business analytics, Docker is, Kafka, The Kubernetes , Design Patterns, Managing the servers, Apache Kafka, The SCALA, ETL/ELT, The data, Project management, Management, Highload, Design of the system, Product development, JPA, Creating a system of linkages, Management of requirements, Time management , Design system, The kubernetes, Docker-compose, Analysis of the requirements, The swagger, Postgres, Technical service, MQ, Technical task, Process management, Optimizing the processes, Kotlin, Developing electrical systems, Analysis of fi

In [29]:
#@title Формирую чанки (Langchain Document)

vacancies_chunks_1 = []
vacancies_chunks_2 = []
vacancies_chunks_3 = []
vacancies_chunks_all = []

for value in vacancies_collection.find(): # по всем вакансиям из базы, коллекции vacancies
    # 1. Position, Skills, M.Requirements, Add.Requirements
    vacancies_chunks_1.append(Document(page_content=value['line_1_en'], metadata={'file': value['_id']}))
    # 2. Levels, WorkFormat, Location, Salary
    vacancies_chunks_2.append(Document(page_content=value['line_2_en'], metadata={'file': value['_id']}))
    # 1 + 3. (3.Project Tasks)
    vacancies_chunks_3.append(Document(page_content=value['line_1_en'] + value['line_3_en'],
                                       metadata={'file': value['_id']}))
    # 1 + 2 + 3
    vacancies_chunks_all.append(Document(page_content=value['line_1_en'] + value['line_2_en'] + value['line_3_en'],
                                         metadata={'file': value['_id']}))

In [ ]:
#@title Индексная (векторная) база по выбранным полям Вкансий
# Подсчет токенов
def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

# количество токенов индексной базы и стоимость. model Ada v2
def tokens_count_db_index(source_chunks):
    count_tokens = 0
    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    count_tokens += count_token
    # Embedding model Ada v2 - $0.10 / 1M tokens - 13/03/2024 - https://openai.com/pricing
    price = 0.1 * count_tokens / 1e6
    print(f'К-во токенов в документе: {count_token}. '
          f'Цена создания индексной базы: $ {price}')

# Если не в колабе, ВКЛЮЧИТЬ VPN
for n, chunks in enumerate([vacancies_chunks_1,      #v1. 1. Position, Skills, M.Requirements, Add.Requirements
                            vacancies_chunks_2,      #v2. 2. Levels, WorkFormat, Location, Salary
                            vacancies_chunks_3,      #v3. 1 + 3. (3.Project Tasks)
                            vacancies_chunks_all]):  #v4. 1 + 2 + 3
    db = FAISS.from_documents(chunks, OpenAIEmbeddings())
    db.save_local(folder_path='../data', index_name=f'db_vacancies_{n+1}_index')

In [30]:
# db = FAISS.from_documents(vacancies_chunks_3, OpenAIEmbeddings())
# db.save_local(folder_path='../data', index_name=f'db_vacancies_3_index')

# Резюме
### Ключевые поля резюме для поиска вакансий
- Позиция "title" + "professional_roles.name"
- Скилы "skill_set"
- Доп. информация от кандидата "skills"
- Зарплата "salary"
- График работы "schedule"
- Локация "area.name"
- Отношение к релокации "relocation"
- Опыт работы "experience.position" и "experience.description"
- Трудовой стаж (месяцев) "total_experience"
- Языки "language"

In [20]:
# Формирую строки из ключевых полей резюме
for value in resumes_collection.find(): # по все резюме из коллекции
    schedules = '' # График работы
    for schedule in value["schedules"]:
        schedules += f"{schedule['name']}, "

    experience = '' # Опыт
    for n, place in enumerate(value['experience']):
        experience += f"Job experience {n+1}: Position - {place['position']}. \
                        Job description {n+1}: {place['description']} "

    languages = '' # Языки
    for lang in value['language']:
        languages += f"{lang['name']} - {lang['level']['name']}. "

    try: # Если указана зарплата
        currency = value['salary']['currency']
        salary = f"Desired salary: {value['salary']['amount']} {currency if currency!='RUR' else 'рублей в месяц'}. "
    except Exception as ex:
        salary = ''

    if value['skills'] != None: # Add.skills
        add_skills = f"Add.skills: {value['skills']}."
    else:
        add_skills = ''


    #r1. Position, Skills, Add.Skills
    resume_line_1 = re.sub(r'\s+', ' ', f"""
                         Position: {value['title']}. {value['professional_roles'][0]['name']}.
                         Skills: {(', ').join(value['skill_set'])}.
                         {add_skills}""")
    #r2. Salary, Languages, Job schedule, Location, Relocation, Total experience
    resume_line_2 = re.sub(r'\s+', ' ', f"""
                         {salary}
                         Languages: {languages}.
                         Job schedule: {schedules}.
                         Location: {value['area']['name']}.
                         Attitude to relocation: {value['relocation']['type']['name']}.
                         Total experience: {value['total_experience']['months']} months. """)
    #r3. Experience (position + description)
    resume_line_3 = re.sub(r'\s+', ' ', f'{experience}')

    try: # сохраняю сформированные строки из полей резюме в коллекцию resumes
        resumes_collection.find_one_and_update({'_id': value['_id']}, # нахожу резюме по _id
                                               {'$set': {'line_first_1': resume_line_1,    # Position, Skills, Add.skills
                                                         'line_first_2': resume_line_2,    # Salary, Lang, Schedule, Location...
                                                         'line_first_3': resume_line_3}})  # Experience (position, description)
    except Exception as ex:
        print(ex)

In [21]:
# Пример строки с выбранными ключевыми полями
ex_resume = resumes_collection.find_one({'_id': 'a293-405b-8e46-f47328bea37a.json'})
print(ex_resume['line_first_1'], '\n')
print(ex_resume['line_first_2'], '\n')
print(ex_resume['line_first_3'])

 Position: Java программист. Программист, разработчик. Skills: ORACLE, Docker, База данных: Oracle, MySQL, REST, XML, СУБД, Atlassian Jira, Gradle, JDBC, Spring, MongoDB, Unix, GitHub, Hibernate, JSON, REST API, HTTP, Unit Testing, Backend, Maven, JPA, JMS, SOAP, Microservices, Java Script, Java EE, Java SE, MS SQL, Linux.  

 Desired salary: 200000 рублей в месяц. Languages: Русский - Родной. Английский - B1 — Средний. . Job schedule: Полный день, Сменный график, Гибкий график, Удаленная работа, . Location: Оренбург. Attitude to relocation: не готов к переезду. Total experience: 364 months.  

Job experience 1: Position - Java developer (удаленно). Job description 1: Разработка приложений с применением Java 8/11/17, Spring (Data, Boot, Security, Web, Cloud Config, AOP), Git / Gitflow / Bitbucket, Jenkins, микросервисы, docker / docker-compose, Maven / Gradle, Oracle / MySQL / PostgreSQL / MS SQL, Hibernate, JBoss / Glassfish / WildFly / Tomcat / Azure, Unix (FreeBSD, RedHat, CentOS), 

# Подбор вакансий (similarity_search_with_score) под резюме

In [11]:
#@title similarity_search_with_score
def ids_and_scores(query, db, k=5):
    docs_and_scores = db.similarity_search_with_score(query, k=k)
    print('\nScores:', [doc[1] for doc in docs_and_scores])
    return [f"{doc[0].metadata['file']}" for doc in docs_and_scores]

# Загружаю ранее сохраненные индексные базы с вакансиями
embed = OpenAIEmbeddings()
#v1. 1.Position, Skills, M.Requirements, Add.Requirements
db_v1 = FAISS.load_local(folder_path='../data', embeddings=embed,
                                  index_name='db_vacancies_1_index')
#v2. 2.Levels, WorkFormat, Location, Salary
db_v2 = FAISS.load_local(folder_path='../data', embeddings=embed,
                                  index_name='db_vacancies_2_index')
#v3. 1 + 3. (3.Project Tasks)
db_v3 = FAISS.load_local(folder_path='../data', embeddings=embed,
                                  index_name='db_vacancies_3_index')
#v4. 1 + 2 + 3
db_v4 = FAISS.load_local(folder_path='../data', embeddings=embed,
                                  index_name='db_vacancies_4_index')

In [22]:
# Любое резюме
ex_resume = resumes_collection.find_one({'_id': 'a293-405b-8e46-f47328bea37a.json'})
#r1. Position, Skills, Add.Skills
r1 = transtator.translate(ex_resume['line_first_1'])
#r2. Salary, Languages, Job schedule, Location, Relocation, Total experience
r2 = transtator.translate(ex_resume['line_first_2'])
#r3. Experience (position + description)
r3 = transtator.translate(ex_resume['line_first_3'][:4999])

In [9]:
#@title Поиск похожестей по разным вариантам полей
# На докальном компе Включить VPN

queries = [r1, r1+r3, r1+r2+r3]
#r1. Position, Skills, Add.Skills
#r2. Salary, Languages, Job schedule, Location, Relocation, Total experience
#r3. Experience (position + description)
queries_names = ['r1', 'r1+r3', 'r1+r2+r3']

dbs = [db_v1, db_v3, db_v4]
#v1. 1.Position, Skills, M.Requirements, Add.Requirements
#v2. 2.Levels, WorkFormat, Location, Salary
#v3. 1 + 3. (3.Project Tasks)
#v4. 1 + 2 + 3
dbs_names = ['v1', 'v3', 'v4']

for nq, query in enumerate(queries):
    for nd, db in enumerate(dbs):
        print(f'\n{queries_names[nq]}. {dbs_names[nd]}.')
        print(ids_and_scores(query, db))
        time.sleep(10)


r1. v1.

Scores: [0.17173511, 0.20397216, 0.2069412, 0.21194778, 0.2474529]
['Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json', 'Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json', 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json', 'Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json', 'Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json']

r1. v3.

Scores: [0.17759915, 0.21428558, 0.21683648, 0.21904373, 0.25692102]
['Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json', 'Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json', 'Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json', 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json', 'Системный_аналитик-5abf0d44-301a-44cd-b520-bd99dc58e154.json']

r1. v4.

Scores: [0.18486986, 0.22332788, 0.22575468, 0.231029, 0.27319628]
['Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json', 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json', 'Java_разработч

In [13]:
# Поля Резюме r1+r3:
# r1. Position, Skills, Add.Skills
# r3. Experience (position + description)
# Поля вакансии v3:
# Position, Skills, M.Requirements, Add.Requirements + Project Tasks

ids = ids_and_scores(r1+r3, db_v3, k=5)
ids


Scores: [0.14696717, 0.16222467, 0.18171781, 0.18187495, 0.21099737]


['Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json',
 'Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json',
 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json',
 'Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json',
 'Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json']

In [14]:
# Все выбранные поля: r1+r2+r3. v4.

ids = ids_and_scores(r1+r2+r3, db_v4, k=5)
ids


Scores: [0.14639908, 0.15628713, 0.15652376, 0.16035888, 0.19916338]


['Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json',
 'Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json',
 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json',
 'Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json',
 'Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json']

# OpenAI

In [18]:
# функция добавления переходов на новую строку для удобства чтения
def format_newlines(text, max_len=100):
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)

# Стоимость запроса + ответ для "gpt-3.5-turbo-0125"
def print_tokens_count(completion):
    # "gpt-3.5-turbo-0125" - Input: $0.50 / 1M tokens - Output: $1.50 / 1M tokens - 07/03/2024 - https://openai.com/pricing
    price = 0.5 * completion.usage.prompt_tokens / 1e6 + 1.5 * completion.usage.completion_tokens / 1e6
    print(f'Использовано токенов: {completion.usage.prompt_tokens} + '
                                f'{completion.usage.completion_tokens} = '
                                f'{completion.usage.total_tokens}. '
                                f'Цена запроса + ответ: $ {price}\n')

def answer_gpt(messages, temp=0.5):
    completion = openai.chat.completions.create(model="gpt-3.5-turbo-0125",
                                                messages=messages,
                                                temperature=temp)
    print_tokens_count(completion)
    return completion.choices[0].message.content

In [24]:
# Резюме: 'a293-405b-8e46-f47328bea37a.json'
test_resume = r1 + r2 + r3
# Выбранная вакасия: 'Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json'
vacancy = vacancies_collection.find_one(
          {'_id': 'Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json'})
test_vacancy = vacancy['line_1_en'] + vacancy['line_2_en'] + vacancy['line_3_en']

system = re.sub(r'\s+', ' ', """You are a professional HR.
    You help candidates (applicants) to get a job for the chosen vacancy""")

message_content = re.sub(r'\s+', ' ', f"""Based on the candidate's resume, you have found a suitable vacancy.
    Candidate's desired position, candidate's skills, additional information about the candidate
    from the resume: {test_resume}. Requirements for the candidate, according
    to the vacancy you have chosen: {test_vacancy}. Analyze the information from the candidate's
    resume and the requirements from the vacancy.""")

question = re.sub(r'\s+', ' ', f"""Answer the questions:
    1. What is the percentage from 0 to 100 of matching skills in the candidate's resume compared
       to the necessary skills in the vacancy?
    2. What skills and competencies does the candidate lack in the resume compared to the necessary
       skills and competencies in the vacancy?
    3. Compare and give a conclusion about the compliance and differences in the resume part: {r2}
       and the vacancy part: {vacancy['line_2_en']}. Compare only what is logical to compare.

    Answer all questions in Russian, with the exception of the words denoting the candidate's skills,
    if you write about them. So, answer in Russian, but if you use words denoting the candidate's skills,
    write skills in English.""")

message = [{"role": "system", "content": system},
           {"role": "user", "content": f"{message_content}\n{question}"}]

response = answer_gpt(message, temp=0.3)
print(format_newlines(response))

Использовано токенов: 1317 + 370 = 1687. Цена запроса + ответ: $ 0.0012135000000000002

 1. Процент совпадения навыков в резюме кандидата с необходимыми навыками в вакансии составляет
 около 70-75%.

 2. Кандидату не хватает следующих навыков и компетенций по сравнению с требованиями вакансии:
 Kotlin, Spring Cloud, *nix command line, log aggregation systems (ELK), Kubernetes, ability to
 analyze performance problems, experience in designing and implementing CI/CD processes on Gitlab,
 experience in tracing complex errors and building cross-service integration, experience working in
 a team using agile development methods specific to FinTech projects.

 3. Сравнение и вывод о соответствии и различиях в части резюме: Желаемая зарплата кандидата
 составляет 200 000 рублей в месяц, в то время как вакансия предлагает 551 040 рублей в месяц. Опыт
 работы кандидата составляет 364 месяца, что соответствует уровню Middle+, Senior в вакансии.
 Кандидат предпочитает работу удаленно, что соответс